# Sprint 1 - Project - 20210121

**Team Pat Members:** Hurly | Joseph | Mikee | Phoemela

-------------------

## Instructions

**Setup**
- Download the files from the compressed folder: education_analytics.zip
- Discuss with your group, what are the questions you want answered? 
- Strategize on how you will distribute the tasks among yourselves. 
- Output

The final output must be a data story involving the insights found in your analysis. Included in the story are:
- Background / Context - What is it you are trying to solve and why did you choose this problem?
- Data Information - What are your sources of data? Which information from the data did you use? How reliable is the data?
- Methodology - What are the tools and models you used? What is your pipeline to arrive at the results?
- EDA - What valuable information did you find from the data?
- Cluster results - What are the characteristics of the clusters you formed?
- Conclusion and Recommendations: (a) Education - what strategy would you recommend based on the clusters formed? (b) Machine Learning - how can you improve your analysis?

Output must be presented with Streamlit deployed in Heroku. The group may choose to use powerpoint to supplement their Streamlit presentation.  

**Presentation**
- Each group will be given 5 minutes to present.
- Each member of the group must be given the opportunity to talk. 
- A 5min questions and answers will be given after each group. 

**Submission**
- Submission is by group
- Each group must submit the following: (a) link to Heroku app (b) link to Github containing the Jupyter Notebook/s used (Notebook must be consolidated and should run from top to bottom without errors) (c) powerpoint, if available (d) csv files used, if different from the provided files
 
**Deadline**
- Initial submission: on or before January 22 (Friday), 8:00 PM
- Final submission: on or before January 23 (Saturday), 6:00 PM 

----------------------

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

pd.set_option('max_colwidth', None)
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
pd.set_option('display.float_format', '{:,.2f}'.format)

----------------------


## Data Preparation

In [2]:
masterlist = pd.read_csv('education_analytics/Masterlist of Schools.csv')
print(masterlist.shape)
print(masterlist.columns)

df_all = masterlist.copy()

files = glob.glob('education_analytics/*.csv')

for file in files:
    if 'Masterlist' not in file:
        try:
            file_df = pd.read_csv(file, encoding = 'utf-8')
        except:
            file_df = pd.read_csv(file, encoding = 'latin-1')
        print(file)
        print(file_df.shape)
        print(file_df.columns)
        if 'School ID' in file_df.columns:
            file_df = file_df.rename(columns = {'School ID':'school.id'})
        df_all = df_all.merge(file_df, on = 'school.id', how = 'left')
        
print(df_all.shape)
print(df_all.columns)
print(df_all.info())

(46603, 23)
Index(['school.id', 'school.name', 'school.region', 'school.region.name',
       'school.province', 'school.cityhall', 'school.division',
       'school.citymuni', 'school.district', 'school.legdistrict',
       'school.type', 'school.abbrev', 'school.previousname',
       'school.mother.id', 'school.address', 'school.established',
       'school.classification', 'school.classification2',
       'school.curricularclass', 'school.organization', 'school.cityincome',
       'school.cityclass', 'school.urban'],
      dtype='object')
education_analytics\Enrollment Master Data_2015_E.csv
(38649, 17)
Index(['School ID', 'Kinder Male', 'Kinder Female', 'Grade 1 Male',
       'Grade 1 Female', 'Grade 2 Male', 'Grade 2 Female', 'Grade 3 Male',
       'Grade 3 Female', 'Grade 4 Male', 'Grade 4 Female', 'Grade 5 Male',
       'Grade 5 Female', 'Grade 6 Male', 'Grade 6 Female', 'SPED NG Male',
       'SPED NG Female'],
      dtype='object')
education_analytics\Enrollment Master Data_201

In [3]:
df_all.head(10)

,school.id,school.name_x,school.region,school.region.name,school.province,school.cityhall,school.division,school.citymuni,school.district,school.legdistrict,school.type,school.abbrev,school.previousname,school.mother.id,school.address,school.established,school.classification,school.classification2,school.curricularclass,school.organization,school.cityincome,school.cityclass,school.urban,Kinder Male,Kinder Female,Grade 1 Male,Grade 1 Female,Grade 2 Male,Grade 2 Female,Grade 3 Male,Grade 3 Female,Grade 4 Male,Grade 4 Female,Grade 5 Male,Grade 5 Female,Grade 6 Male,Grade 6 Female,SPED NG Male_x,SPED NG Female_x,Grade 7 Male,Grade 7 Female,Grade 8 Male,Grade 8 Female,Grade 9 Male,Grade 9 Female,Grade 10 Male,Grade 10 Female,Grade 11 Male,Grade 11 Female,Grade 12 Male,Grade 12 Female,SPED NG Male_y,SPED NG Female_y,school.name_y,school.enrollment,school.offering,school.mooe,rooms.standard.academic,rooms.standard.unused,rooms.nonstandard.academic,rooms.nonstandard.unused,School Name,Region,Province,Municipality,Division,District,Offering,Name of Principal,Enrolment,Latitude,Longitude,teachers.instructor,teachers.mobile,teachers.regular,teachers.sped
0,101746,"A. Diaz, Sr. ES",Region I,Ilocos Region,PANGASINAN,PANGASINAN,"Pangasinan II, Binalonan",BAUTISTA,Bautista,5th District,School with no Annexes,ADSES,none,"101,746.00","Brgy. Dias Bautista, Pang",1/1/1930,Elementary,DepED Managed,Elementary,Monograde,P 25 M or more but less than P 35 M,NaN,Partially Urban,57.00,45.00,56.00,58.00,63.00,61.00,55.00,61.00,62.00,55.00,61.00,48.00,48.00,51.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"A. Diaz, Sr. ES",781.00,Elementary,"341,000.00",15.00,0.00,0.00,0.00,"A. Diaz, Sr. ES",Region I,Pangasinan,Bautista,"Pangasinan II, Binalonan",Bautista,ES,Teresita B. Cabrera,781.00,15.80,120.50,0.00,1.00,20.00,0.00
1,102193,A. P. Santos ES (SPED Center),Region I,Ilocos Region,ILOCOS NORTE,ILOCOS NORTE,Laoag City,LAOAG CITY (Capital),Laoag City District II,1st District,School with no Annexes,APSES,-,"102,193.00",A.G. Tupaz,1/1/1944,Elementary,DepED Managed,Kinder & Elementary,Monograde,P 240 M or more but less than P 320 M,Component City,Partially Urban,20.00,24.00,38.00,30.00,37.00,18.00,29.00,33.00,23.00,41.00,39.00,27.00,26.00,26.00,32.00,22.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A. P. Santos ES (SPED Center),465.00,Elementary,"275,000.00",13.00,3.00,0.00,0.00,A. P. Santos ES (SPED Center),Region I,Ilocos Norte,Laoag City (Capital),Laoag City,Laoag City District II,ES,"Christine D. Alipio, Ed.D.",465.00,18.20,120.59,0.00,2.00,12.00,4.00
2,101283,A.P. Guevara IS,Region I,Ilocos Region,PANGASINAN,PANGASINAN,"Pangasinan I, Lingayen",BAYAMBANG,Bayambang II,3rd District,School with no Annexes,NaN,A.P. Guevarra IS,"101,283.00",-,1/1/1945,Elementary,DepED Managed,Elementary,Monograde,P 55 M or more,NaN,Partially Urban,16.00,11.00,11.00,10.00,9.00,13.00,15.00,13.00,14.00,11.00,23.00,13.00,7.00,9.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.P. Guevara IS,175.00,Elementary,"132,000.00",7.00,0.00,0.00,0.00,A.P. Guevara IS,Region I,Pangasinan,Bayambang,"Pangasinan I, Lingayen",Bayambang II,ES,Maximo F. Jimenez,175.00,15.78,120.46,0.00,1.00,6.00,0.00
3,100216,Ab-Abut ES,Region I,Ilocos Region,ILOCOS NORTE,ILOCOS NORTE,Ilocos Norte,PIDDIG,Piddig,1st District,School with no Annexes,AES,-,"100,216.00",Sucsuquen,1/1/1964,Elementary,DepED Managed,Elementary,Monograde,P 35 M or more but less than P 45 M,NaN,Partially Urban,10.00,6.00,10.00,17.00,22.00,11.00,22.00,12.00,12.00,11.00,16.00,7.00,13.00,10.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ab-Abut ES,179.00,Elementary,"135,000.00",6.00,0.00,0.00,0.00,Ab-Abut ES,Region I,Ilocos Norte,Piddig,Ilocos Norte,Piddig,ES,Melanio P. Tomas,179.00,18.17,120.70,0.00,1.00,6.00,0.00
4,100043,Abaca ES,Region I,Ilocos Region,ILOCOS NORTE,ILOCOS NORTE,Ilocos Norte,BANGUI,Bangui,1st District,School with no Annexes,AES,none,"100,043.00",None

In [4]:
df_all = df_all.drop(columns=['school.name_x','school.name_y', 'school.previousname', 'school.mother.id', 'school.address', 'school.cityclass', 'school.offering' ,'Region', 'Province',
       'Municipality', 'Division', 'District', 'Offering', 'Name of Principal', 'Enrolment'])

df_all = df_all[df_all['school.classification']=='Elementary']

df_all.shape

(38689, 61)

### For feature engineering

- add enrollees for all year level columns - Sef
- total male columns - Pat 
- total female columns - Mikee
- add all rooms column - Hurly
- add all teachers column - Phoemela

- **student teacher ratio** - total enrollees over total teachers - Sef
- **mooe per student** - mooe over total enrollees - Hurly
- **student per room** - total enrollees over all rooms - Mikee
- **sped ratio** - total sped, sped over total enrollees - Phoemela
- **female to male ratio** - female over total enrollees - Pat

In [5]:
df_all['enrollees'] = df_all[df_all.columns[df_all.columns.str.contains('Male|Female')]].sum(axis=1)
# df_all['enrollees'] = df_all[['Kinder Male', 'Kinder Female', 'Grade 1 Male','Grade 1 Female','Grade 2 Male','Grade 2 Female','Grade 3 Male','Grade 3 Female','Grade 4 Male','Grade 4 Female','Grade 5 Male','Grade 5 Female','Grade 6 Male','Grade 6 Female','SPED NG Male_x','SPED NG Female_x']].sum(axis=1)
df_all['females']= df_all['Kinder Female'] + df_all['Grade 1 Female'] + df_all['Grade 2 Female'] + df_all['Grade 3 Female'] + df_all['Grade 4 Female'] + df_all['Grade 5 Female']+ df_all['Grade 6 Female'] + df_all['SPED NG Female_x']
df_all['males'] = df_all[df_all.columns[df_all.columns.str.contains('Male')]].sum(axis = 1)
df_all["allRooms"] = df_all["rooms.standard.academic"] + df_all["rooms.standard.unused"] + df_all["rooms.nonstandard.academic"] + df_all["rooms.nonstandard.unused"]
df_all['teachers'] = df_all['teachers.instructor'] + df_all['teachers.mobile'] + df_all['teachers.regular'] + df_all['teachers.sped']
df_all['total_enrollees_sped'] = df_all['SPED NG Male_x'] + df_all['SPED NG Female_x']
df_all[" school.mooe "] = df_all[" school.mooe "].str.replace(",","").astype(float)

df_all = df_all[(df_all[' school.mooe ']!=0) & (df_all['enrollees']!=0) & (df_all['teachers']!=0) & (df_all['allRooms']!=0)]

df_all['student_teacher_ratio'] = df_all['enrollees'] / df_all['teachers']
df_all["mooe_student_ratio"] = df_all[" school.mooe "]/df_all["enrollees"]
df_all['student_per_rm'] = df_all['enrollees']/df_all['allRooms']
df_all['sped_ratio'] = df_all['total_enrollees_sped']/df_all['enrollees']
df_all['female_to_male'] = df_all['females'] / df_all['enrollees']

### Data checks

Particularly for st ratio, mooe per student, student per room, sped ratio, female to male ratio columns,

- Are there missing values? - Sef
- Are there zeroes? - Mikee
- Are there duplicates? - Phoemela
- Are there outliers? - Hurly

How do we deal with these data? Just check number of zeroes per relevant column, all the rest (dupes, missing, outliers) drop.

In [6]:
df_all.isna().sum()

school.id                         0
school.region                     0
school.region.name                0
school.province                   0
school.cityhall                   0
school.division                   0
school.citymuni                   0
school.district                   0
school.legdistrict                0
school.type                       0
school.abbrev                 24212
school.established                3
school.classification             0
school.classification2            0
school.curricularclass            0
school.organization               0
school.cityincome               769
school.urban                      2
Kinder Male                       0
Kinder Female                     0
Grade 1 Male                      0
Grade 1 Female                    0
Grade 2 Male                      0
Grade 2 Female                    0
Grade 3 Male                      0
Grade 3 Female                    0
Grade 4 Male                      0
Grade 4 Female              

In [7]:
df_all = df_all.dropna(subset = ['student_teacher_ratio','student_per_rm','sped_ratio','female_to_male','mooe_student_ratio'])

In [8]:
df_all.shape

(35848, 72)

In [9]:
df_all[df_all["school.id"].duplicated(keep=False)].sort_values(by="school.id")

,school.id,school.region,school.region.name,school.province,school.cityhall,school.division,school.citymuni,school.district,school.legdistrict,school.type,school.abbrev,school.established,school.classification,school.classification2,school.curricularclass,school.organization,school.cityincome,school.urban,Kinder Male,Kinder Female,Grade 1 Male,Grade 1 Female,Grade 2 Male,Grade 2 Female,Grade 3 Male,Grade 3 Female,Grade 4 Male,Grade 4 Female,Grade 5 Male,Grade 5 Female,Grade 6 Male,Grade 6 Female,SPED NG Male_x,SPED NG Female_x,Grade 7 Male,Grade 7 Female,Grade 8 Male,Grade 8 Female,Grade 9 Male,Grade 9 Female,Grade 10 Male,Grade 10 Female,Grade 11 Male,Grade 11 Female,Grade 12 Male,Grade 12 Female,SPED NG Male_y,SPED NG Female_y,school.enrollment,school.mooe,rooms.standard.academic,rooms.standard.unused,rooms.nonstandard.academic,rooms.nonstandard.unused,School Name,Latitude,Longitude,teachers.instructor,teachers.mobile,teachers.regular,teachers.sped,enrollees,females,males,allRooms,teachers,total_enrollees_sped,student_teacher_ratio,mooe_student_ratio,student_per_rm,sped_ratio,female_to_male


In [10]:
df_all.shape

(35848, 72)

In [13]:
engineeredData = df_all[["allRooms","enrollees","males","females","teachers","student_per_rm",
             "mooe_student_ratio","sped_ratio","student_teacher_ratio","female_to_male"]]

Q1 = engineeredData.quantile(0.25)
Q3 = engineeredData.quantile(0.75)
IQR = Q3-Q1

df_all_no_outlier = df_all[(engineeredData >= Q1 - 1.5*IQR) & 
                           (engineeredData <= Q3 + 1.5*IQR)]

In [15]:
df_all_no_outlier.shape

(35848, 72)

In [16]:
df_final = df_all_no_outlier.copy()

In [17]:
df_features = df_final[["student_per_rm","mooe_student_ratio","sped_ratio","student_teacher_ratio","female_to_male"]]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_features)

### Modelling

- KMeans
- Hierarchical (single, average, complete, ward)